In [4]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import ssl
import re

number_of_pages = 45

restaurants = []
infos = []
addresses = []
checks = []

headers = {'User-Agent': 'Mozilla/5.0',
'cookie': 'split.rkSplit=false; countryFrom=kz; project=68; _ga=GA1.2.809907238.1551132602;'
          '_ym_uid=1551132602418148161; _ym_d=1551132602; _2gis_webapi_session=7b3a262e-3de3-4d68-92da-7c2f54752ced; '
          '_2gis_webapi_user=412ec892-87a6-4bc6-9b11-b3ec982933f3; socialRefererSet=1; _gid=GA1.2.2117315879.1551816480; '
          'regionsHash=a5ceb3915d34ded7ef8e1295d4d1b430; _ym_isad=1; '
          'ipp_key=v1551868467964%2Fv3394bde41e540e2206ba54163aeca6afa04ab3%2FJ9QfGc%2fX85cnC5yw80L6eg%3d%3d; '
          'ipp_uid=1551868467963%2fRifwXqSRLKAk4cl1%2fhjKqVc9ZndV5Swv6RbTT8g%3d%3d; ipp_uid1=1551868467963; '
          'ipp_uid2=RifwXqSRLKAk4cl1%2fhjKqVc9ZndV5Swv6RbTT8g%3d%3d; ipp_uid_tst=; ipp_static_key=; _2gis_wsize=734x1002'}

for i in range(1, number_of_pages + 1):
  print("processing page " + str(i) + " ... ")
  
  url = "https://2gis.kz/almaty/search/%D0%B5%D0%B4%D0%B0/page/{}?queryState=center%2F76.945152%2C43.23845%2Fzoom%2F12".format(i)
  request = Request(url, headers=headers)

  source = urlopen(request, context=ssl.SSLContext()).read()
  soup = BeautifulSoup(source, features="html.parser")

  content = soup.select(".searchResults__list > article > .miniCard__content")

  restaurants.extend([x.text for x in soup.select(".searchResults__list > article > .miniCard__content > header > h3")])

  for d in content:
    info = d.select(".miniCard__micro") 
    if (len(info) == 0):
      infos.append(None)
    else:
      infos.append(info[0].text)
      
  for d in content:
    address = d.select(".miniCard__desc > .miniCard__address") 
    if (len(address) == 0):
      addresses.append(None)
    else:
      addresses.append(address[0].text)
      
  for d in content:
    check = d.select(".miniCard__additional > .miniCard__attrs > .miniCard__attrsItem") 
    if (len(check) == 0 or 'тнг' not in check[0].text):
      checks.append(0)
    else:
      checks.append(int(re.findall("[0-9]+ тнг", check[0].text)[0].split(" тнг")[0]))

  print ("Processed : restaurants({}), infos({}), addresses({}), checks({})"
    .format(len(restaurants), len(infos), len(addresses), len(checks)))

data = { 'restaurants': restaurants,
   'infos': infos,
   'addresses': addresses,
   'checks': checks}

print ("Creating data frame ...")
df = pd.DataFrame(data=data)
print(df.head())

print ("Exporting data to excel ...")
df.to_excel(("output.xlsx"))

processing page 1 ... 


HTTPError: HTTP Error 403: Forbidden